# CNN Baseline - iWildCam 2019

In [ ]:
import os
import json
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub


from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [ ]:
PATH="../input/iwildcam-2019-fgvc6/"
PATH

## Loading the dataset

In [ ]:
path_to_zip = "../input/iwildcam-2019-fgvc6/train_images.zip"
directory_to_extract="../output/kaggle/working/train_images"

with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract)

In [ ]:
train_image_files = list(os.listdir(os.path.join(directory_to_extract)))
print("Number of image files: train:{}".format(len(train_image_files)))

In [ ]:
classes_wild = {0: 'empty', 1: 'deer', 2: 'moose', 3: 'squirrel', 4: 'rodent', 5: 'small_mammal', \
                6: 'elk', 7: 'pronghorn_antelope', 8: 'rabbit', 9: 'bighorn_sheep', 10: 'fox', 11: 'coyote', \
                12: 'black_bear', 13: 'raccoon', 14: 'skunk', 15: 'wolf', 16: 'bobcat', 17: 'cat',\
                18: 'dog', 19: 'opossum', 20: 'bison', 21: 'mountain_goat', 22: 'mountain_lion'}

In [ ]:
train_df = pd.read_csv(os.path.join(PATH, 'train.csv'))
#display(train_df.head())


In [ ]:
train_df['classes_wild'] = train_df['category_id'].apply(lambda cw: classes_wild[cw])

In [ ]:
train_df.head(2)

## Create Early Stopping Callback 

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

## Creating and Training the Model

In [ ]:
## dataset splitting
x_train, x_test = train_test_split(train_df, test_size=0.3, random_state=42)
print('x_train dimension: {}, x_test dimension: {}'.format(x_train.shape, x_test.shape))
  

In [ ]:
test_datagen =tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                 validation_split=0.25,
                                 horizontal_flip = True,
                                 vertical_flip = True,                                           
                                 zoom_range = 0.3,
                                 width_shift_range = 0.3,
                                 height_shift_range=0.3
                                )

In [ ]:
batch_size = 150
num_classes = 14
epochs = 30

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
                    dataframe=x_train,
                    directory="../output/kaggle/working/train_images/",
                    x_col="file_name",
                    y_col="classes_wild",
                    subset="training",
                    batch_size=batch_size,
                    seed=424,
                    shuffle=True,
                    class_mode="categorical",
                    target_size=(64,64))

valid_generator=train_datagen.flow_from_dataframe(
                    dataframe=x_train,
                    directory="../output/kaggle/working/train_images/",
                    x_col="file_name",
                    y_col="classes_wild",
                    subset="validation",
                    batch_size=batch_size,
                    seed=424,
                    shuffle=True,
                    class_mode="categorical",
                    target_size=(64,64))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64,64,3)),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    #tf.keras.layers.GlobalAveragePooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    

])

model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)


history = model.fit_generator(generator=train_generator,  
                                    steps_per_epoch=100,
                                    validation_data=valid_generator, 
                                    validation_steps=70,
                                    epochs=30,
                                    callbacks = [early],
                                    )

## Evaluation

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
display_training_curves(history.history['acc'], history.history['val_acc'], 'accuracy', 212)

## Prediction on Test Dataset

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
                    dataframe=x_test,
                    directory="../output/kaggle/working/train_images/",
                    x_col="file_name",
                    y_col="classes_wild",
                    batch_size=batch_size,
                    seed=424,
                    shuffle=True,
                    class_mode="categorical",
                    target_size=(64, 64))

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=32)
print('test_loss: {} and test_acc: {} '.format(test_loss, test_acc))

## Save as TFLite Model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
cnn_baseline_model = converter.convert()

In [ ]:
model_name = "cnn_64_f_baseline_model"
open(f"{model_name}.tflite" , "wb").write(cnn_64_f_baseline_model)